In [1]:
# Imports
import pandas as pd
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
import time

# Global Variables 
end_complete=r'\*1\*1'
end_complete_esc='*1*1'
delimiter_000 = r'\*000\*'
delimiter_000_esc = r'\*000\*'
delimiter_00 = r'\*00\*'

In [150]:
# Functions


# functions to get offset
def getTripplezero(string):
    
    split_string = string.split(delimiter_000)
    curent_string = split_string[len(split_string)-1]

    return curent_string

def getDoublezero(string):
    
    split_string = string.split(delimiter_00)
    curent_string = split_string[len(split_string)-1]

    return curent_string

def getTimestamp():
    from datetime import datetime
    timestamp = datetime.now()

    return timestamp

In [2]:
# DB Access initialisation
password = "123456@Ab"
encode_pass = quote_plus(password)
db_url = "mysql+mysqlconnector://{USER}:{PWD}@{HOST}/{DBNAME}"
source_db_url = db_url.format(USER="root", PWD=encode_pass, HOST="localhost:3306", DBNAME='massnet')
source_engine = create_engine(source_db_url, echo=False)

In [3]:
#  filter data to fetch
query = "Select *, length(user_input) as 'string_length',  length(user_input)-length(REPLACE(user_input, '*', '')) AS 'no_of_asterisks' from ussd_audit_logs where is_user = 1 and user_input <> '' and user_input like'4*%' having string_length > 9"

query_11 = "Select *, length(user_input) as 'string_length',  length(user_input)-length(REPLACE(user_input, '*', '')) AS 'no_of_asterisks' from ussd_audit_logs where is_user = 1 and user_input <> '' and user_input like'%*4*%' having string_length > 9"

incomplete_distribution_query = "Select *, length(user_input) as 'string_length',  length(user_input)-length(REPLACE(user_input, '*', '')) AS 'no_of_asterisks' from ussd_audit_logs where is_user = 1 and user_input <> '' and (user_input like'%*4*%' or user_input like'4*%') and user_input not like '%*1*1' having string_length >= 9"

# Household registration , unsuccessful and incomplete  
HH_REG_UNS = ''
HH_REG_INC = ''

# Nets distribution , unsuccessful and incomplete
NETS_DIST_UNS = ''
NETS_DIST_INC = ''


query_cond = query.format(Q_CONDITION=HH_REG_INC)

In [4]:
# Read data

try:
    df_distribution = pd.read_sql(query_cond, source_engine)
    distribution_details = pd.read_sql('select * from distribution_details', source_engine)
    house_holds = pd.read_sql('select * from house_holds_mass_nets', source_engine)
except Exception as e:
    print(f'Could not read data, encountered the following error:  {e}')

In [54]:
# Read data 2

try:
    admin_units = pd.read_sql('select * from administrative_units', source_engine)
except Exception as e:
    print(f'Could not read data, encountered the following error:  {e}')

In [ ]:
# Read data 2

try:
    distr_start_mid_session = pd.read_sql(query_11, source_engine)
    incomplete_distribution_query_data = pd.read_sql(incomplete_distribution_query, source_engine)
except Exception as e:
    print(f'Could not read data, encountered the following error:  {e}')

In [ ]:
# Merging distribution details and households to enable filtering out

dd_hhmn = distribution_details.merge(house_holds, how='left', left_on='hh_id', right_on='id')
# df_distribution.columns

In [ ]:
# Handling filtering

# result = df_distribution.groupby(['no_of_asterisks', 'string_length'])['id'].count().reset_index(name='COUNT(id)')
# filters_result = df_distribution[df_distribution['no_of_asterisks'] == 3 ]

passed = df_distribution[df_distribution['user_input'].str.endswith(end_complete)]
passed_3_asterisks=passed[passed['no_of_asterisks']==3]


incomplete_distribution = df_distribution[~df_distribution['user_input'].str.endswith('1*1')]
complete_distribution = df_distribution[df_distribution['user_input'].str.endswith('1*1')]

In [ ]:
# Handling 3 steps
def handle3steps(payload):
    df_distribution = payload
    passed = df_distribution[df_distribution['user_input'].str.endswith(end_complete_esc)]
    passed_3_asterisks=passed[passed['no_of_asterisks']==3]
    # Explode the column_with_data using the '*' delimiter
    df_exploded = passed_3_asterisks['user_input'].str.split('*', expand=True)

    # Rename the new columns if needed
    df_exploded.columns = [f'part_{i}' for i in range(1, df_exploded.shape[1] + 1)]

    # Concatenate the exploded DataFrame with the original DataFrame
    passed_3_asterisks_result = pd.concat([passed_3_asterisks, df_exploded], axis=1)
    # Output for all the distribution logs with 3 steps - optimum
    # passed_3_asterisks_result.rename(columns={'part_1': 'Step one', 'part_2': 'id_no', 'part_3': 'val_1', 'part_4': 'val_2'})
    passed_3_asterisks_result.to_sql('query_audit',source_engine, if_exists='replace')

In [ ]:
# Handling all cases with 000
def handle3Zero(payload):
    df_distribution = payload
    to_step_one_df = df_distribution[df_distribution['user_input'].str.contains(delimiter_000)]
    to_step_one_df_split = to_step_one_df['user_input'].str.split(delimiter_000).str[-1]
    # to_step_one_df_explode = to_step_one_df_split['user_input'].str.split('*', expand=True)
    to_step_one_df_split_frame = to_step_one_df_split.to_frame(name='user_input_2')
    to_step_one_df_split_frame
    df_exploded_000 = to_step_one_df_split_frame['user_input_2'].str.split('*', expand=True)

    # Rename the new columns if needed
    # to_step_one_df_split.columns = [f'part_{i}' for i in range(1, df_exploded.shape[1] + 1)]

    # Concatenate the exploded DataFrame with the original DataFrame
    df_exploded_000_passed = pd.concat([to_step_one_df, df_exploded_000], axis=1)
    df_exploded_000_passed.to_csv('output/handle3Zero.csv')

    pd_000 = pd.read_excel('output/with 000 .xlsx', sheet_name='upload')
    # pd_000.to_sql('query_audit',source_engine, if_exists='append')

In [ ]:
# Handling all cases with 00
def handleDoubleZero(payload):
    one_step_back = df_distribution[df_distribution['user_input'].str.contains(delimiter_00) & ~df_distribution['user_input'].str.contains(delimiter_000) & df_distribution['user_input'].str.endswith('1*1')]
    one_step_back_exploded = one_step_back['user_input'].str.split('*', expand=True)
    one_step_back_exploded_merged = pd.concat([one_step_back, one_step_back_exploded], axis=1)
    one_step_back_exploded_merged
    one_step_back_exploded_merged.to_csv('output/onestepback no all steps back completed_.csv',index=False)

    pd_00 = pd.read_excel('output/onestepback no all steps back completed.xlsx', sheet_name='upload')
    # pd_00.to_sql('query_audit',source_engine, if_exists='append')

In [ ]:
# Handles the distribution which starts in the mid session 
 
completed_distr_start_mid_session = distr_start_mid_session[distr_start_mid_session['user_input'].str.endswith(end_complete_esc)]

completed_distr_start_mid_session_split = completed_distr_start_mid_session['user_input'].str.split(r'\*4\*').str[-1]
completed_distr_start_mid_session_split_frame = completed_distr_start_mid_session_split.to_frame(name='user_input_2')
completed_distr_start_mid_session_split_frame
completed_distr_start_mid_session_split_frame_exploded = completed_distr_start_mid_session_split_frame['user_input_2'].str.split('*', expand=True)

completed_distr_start_mid_session_split_frame_exploded_passed = pd.concat([completed_distr_start_mid_session, completed_distr_start_mid_session_split_frame_exploded], axis=1)
# completed_distr_start_mid_session_split_frame_exploded_passed.to_csv('output/completed_distr_start_mid_session_split_frame_exploded_passed.csv')
pd_start_middle = pd.read_excel('output/completed_distr_start_mid_session_split_frame_exploded_passed.xlsx', sheet_name='upload')
pd_start_middle.count()
# pd_start_middle.to_sql('query_audit',source_engine, if_exists='append')

In [ ]:
incomplete_distribution_query_data.to_csv('temp/incomplete distribution >= 9 string len.csv')


In [ ]:
# Handles Incomplete Merge which start with step 4 distribution

start_with_4 = incomplete_distribution_query_data[incomplete_distribution_query_data['user_input'].str.startswith(r'4*')]
start_with_4.count()
start_with_4_exploaded = start_with_4['user_input'].str.split('*', expand=True)
start_with_4_merge = pd.concat([start_with_4, start_with_4_exploaded], axis=1)
start_with_4_merge.to_csv('output/incomplete_start_with_4_merge.csv')

In [ ]:
# Handles distribution which start from a start over i.e. 000 then 4


in_4_middle = incomplete_distribution_query_data[incomplete_distribution_query_data['user_input'].str.contains(r'\*4\*')]
in_4_middle.count()
in_4_middle_split = in_4_middle['user_input'].str.split(r'\*4\*').str[-1]
in_4_middle_split_frame = in_4_middle_split.to_frame(name='user_input_2')
in_4_middle_split_frame
in_4_middle_split_frame_explod = in_4_middle_split_frame['user_input_2'].str.split('*', expand=True)


in_4_middle_split_frame_explod_merge = pd.concat([in_4_middle, in_4_middle_split_frame_explod], axis=1)
in_4_middle_split_frame_explod_merge.to_csv('output/in_4_middle_split_frame_explod_merge.csv')

In [5]:
pd_00 = pd.read_excel('output/in_4_middle_split_frame_explod_merge.xlsx', sheet_name='upload')
pd_00.to_sql('incomplete_audit_2',source_engine, if_exists='replace')


pd_00 = pd.read_excel('output/incomplete_start_with_4_merge.xlsx', sheet_name='upload')
pd_00.to_sql('incomplete_audit_2',source_engine, if_exists='append')

252132

In [123]:
read_audits = pd.read_sql('select * from incomplete_audit_2', source_engine)
query_audit = pd.read_sql('select * from query_audit', source_engine)

In [140]:
read_audit = read_audits[~read_audits['part_2'].isin(query_audit['part_2'].astype(int))]

In [143]:
merge_dd_hh = house_holds.merge(distribution_details, how='left', left_on='id', right_on='hh_id')

In [144]:
audit_ids = pd.DataFrame(read_audit['part_2'])
unique_id_audit = read_audit['part_2'].astype(str)

incomplete = merge_dd_hh[~merge_dd_hh['id_no'].isin(unique_id_audit)]

In [145]:
incomplete['no_of_nets'].sum()

unique_id_audit = read_audit['part_2'].astype(str)

jane = merge_dd_hh[merge_dd_hh['id_no'].isin(unique_id_audit)]

In [146]:
merge_au = jane.merge(admin_units, how='left', left_on='location_id', right_on='village_id')

In [147]:
homabay = merge_au[merge_au['county'] == 'Homa Bay County']

In [109]:
homabay['actual_allocation'].sum()- homabay['no_of_nets'].sum()
# hm = homabay[homabay['no_of_nets'].isna()]

# hm['no_of_nets'].unique()

homabay_not  = pd.read_csv('reports/homabay distribution.csv', dtype='str')


In [115]:
helo = homabay_not[homabay_not['id_no'].isin(unique_id_audit)]
# type(homabay_not)

In [118]:
sum = helo['original_allocation'].astype(int)

In [152]:
read_audits['part_2'].unique()
query_audit['part_2'].unique()

homabay.to_csv(f'output/homabay_incomplete{getTimestamp()}.csv')